In [1]:
import pandas as pd 
import warnings
warnings.simplefilter('ignore')

import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

In [2]:
data=pd.read_csv('heart.csv')
data.head()

,Age,Gender,Chest Pain Type,Resting Blood Pressure,Serum Cholesterol,Fasting Blood Sugar,Resting Electrocardiographic Results,Maximum Heart Rate Achieved,Exercise induced angina,Depression induced by exercise relative to rest,Slope of the Peak Exercise ST Segment,Number of Major Vessels Colored by Fluoroscopy,Thalassemia,Target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Age                                              1025 non-null   int64  
 1   Gender                                           1025 non-null   int64  
 2   Chest Pain Type                                  1025 non-null   int64  
 3   Resting Blood Pressure                           1025 non-null   int64  
 4   Serum Cholesterol                                1025 non-null   int64  
 5   Fasting Blood Sugar                              1025 non-null   int64  
 6   Resting Electrocardiographic Results             1025 non-null   int64  
 7   Maximum Heart Rate Achieved                      1025 non-null   int64  
 8   Exercise induced angina                          1025 non-null   int64  
 9   Depression induced by exercise

Age
Gender
Chest Pain Type
Resting Blood Pressure
Serum Cholesterol
Fasting Blood Sugar
Resting Electrocardiographic Results
Maximum Heart Rate Achieved
Exercise induced angina
Depression induced by exercise relative to rest
Slope of the Peak Exercise ST Segment
Number of Major Vessels Colored by Fluoroscopy
Thalassemia
Target

In [4]:
data.isnull().any()

Age                                                False
Gender                                             False
Chest Pain Type                                    False
Resting Blood Pressure                             False
Serum Cholesterol                                  False
Fasting Blood Sugar                                False
Resting Electrocardiographic Results               False
Maximum Heart Rate Achieved                        False
Exercise induced angina                            False
Depression induced by exercise relative to rest    False
Slope of the Peak Exercise ST Segment              False
Number of Major Vessels Colored by Fluoroscopy     False
Thalassemia                                        False
Target                                             False
dtype: bool

In [5]:
data.duplicated().sum()

723

In [6]:
data.shape

(1025, 14)

In [7]:
num_col=[feature for feature in data.columns if data[feature].dtype!='O']

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
x=data.drop(columns='Target',axis=1)
y=data['Target']

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state = 20)

In [11]:
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)

x_test  = scaler.transform(x_test)

In [12]:
models={
    'Logistic Regression':LogisticRegression(),
    'Naive Bayes':GaussianNB(),
    'Random Forest Classfier':RandomForestClassifier(n_estimators=20, random_state=12,max_depth=5),
    'XG Boost':XGBClassifier(learning_rate=0.01, n_estimators=25, max_depth=15,gamma=0.6, subsample=0.52,colsample_bytree=0.6,seed=27, 
                    reg_lambda=2, booster='dart', colsample_bylevel=0.6, colsample_bynode=0.5),
    'K Nearest Neighbors':KNeighborsClassifier(n_neighbors=10),
    'Decision Tree':DecisionTreeClassifier(criterion = 'entropy',random_state=0,max_depth = 6),
    'Support Vector Machine':SVC(kernel='rbf', C=2)
}

In [13]:
model_list = []
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

for model_name, model in models.items():

    model.fit(x_train, y_train)
    y_pred_train = model.predict(x_train)
    y_pred = model.predict(x_test)

    # Evaluate the model
    #train_accuracy = accuracy_score(x_train,y_pred_train)
    test_accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred)

    print(f"Model: {model_name}")
    print("Testing Accuracy: ", test_accuracy)
    print("Precision: ",precision)
    print("Recall: ",recall)
    print("F1 Score: ",f1)
    print("Confusion Matrix:\n ",confusion_mat)


    model_list.append(model_name)
    accuracy_list.append(test_accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)

    print("=" * 35)
    print('\n')

max_accuracy_index = accuracy_list.index(max(accuracy_list))
print(f"The best model based on accuracy is {model_list[max_accuracy_index]} with Testing Accuracy: {accuracy_list[max_accuracy_index]}")

Model: Logistic Regression
Testing Accuracy:  0.7951219512195122
Precision:  0.7622950819672131
Recall:  0.8773584905660378
F1 Score:  0.8157894736842105
Confusion Matrix:
  [[70 29]
 [13 93]]


Model: Naive Bayes
Testing Accuracy:  0.8
Precision:  0.782608695652174
Recall:  0.8490566037735849
F1 Score:  0.8144796380090498
Confusion Matrix:
  [[74 25]
 [16 90]]


Model: Random Forest Classfier
Testing Accuracy:  0.8878048780487805
Precision:  0.8547008547008547
Recall:  0.9433962264150944
F1 Score:  0.8968609865470852
Confusion Matrix:
  [[ 82  17]
 [  6 100]]


Model: XG Boost
Testing Accuracy:  0.848780487804878
Precision:  0.8151260504201681
Recall:  0.9150943396226415
F1 Score:  0.8622222222222222
Confusion Matrix:
  [[77 22]
 [ 9 97]]


Model: K Nearest Neighbors
Testing Accuracy:  0.8731707317073171
Precision:  0.8703703703703703
Recall:  0.8867924528301887
F1 Score:  0.8785046728971962
Confusion Matrix:
  [[85 14]
 [12 94]]


Model: Decision Tree
Testing Accuracy:  0.93170731707